In [1]:
import xarray as xr
import geopandas as gp
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import numpy as np
import os
import json
from compliance_checker.runner import ComplianceChecker, CheckSuite
from collections import defaultdict

In [2]:
## this is taken from code code by Paola Petrelli
def report_test(data, test, files, levels, scores):
    """Report resuls for one kind of test
       
       Input:
           data - json data from checker report as read from file
           test - kind of test to write a report for
           files - a json data reporting scores and warnings for each file
           levels - list of priority levels
           scores - a dictionary to collect each file scores
       Return:
           scores - same as input after updating with the test scores 
           warnings - a dictionary collecting all the warnings by priority level
    """
    #For each file collect scores in scores dictionary
    # and group warnings by priority levels in another dictionary

    print(f"Results for {test} checks")
    print(f"{len(files)} files were checked\n")
    warnings = {l: {} for l in levels}
    for fpath in files:
        fname = fpath.split("/")[-1]
        header = data[fpath][test]['scoreheader']
        header['scored_points'] = data[fpath][test]['scored_points']
        header['possible_points'] = data[fpath][test]['possible_points']
        scores[fpath][test] = header
        for level in levels:
            for w in data[fpath][test][level + "_priorities"]:
                name = w['name'] + ": "
                for m in w['msgs']:
                    try:
                        warnings[level][name+m].append(fname)
                    except:
                        warnings[level][name+m] = [fname]
    for level in levels:
        print(f'{level.capitalize()} priority results\n')
        for m in warnings[level].keys():
            nfiles = len(warnings[level][m])
            print(f'{nfiles} files failed:\n{m}')
            if nfiles < len(files):
                print(f'{[w for w in warnings[level][m]]}\n')
            else:
                print()
    return scores

In [39]:
filelist=['/g/data/wp00/data/QDC-CMIP6/BARRA-R2/ACCESS-CM2/ssp126/r4i1p1f1/day/sfcWind/2035-2064/sfcWind_day_ACCESS-CM2_ssp126_r4i1p1f1_AUS-11_20350101-20351231_qdc-additive-monthly-q100-linear_BARRA-R2-19850101-20141231_historical-19850101-20141231.nc']

In [40]:
check_suite = CheckSuite()
check_suite.load_all_available_checkers()
ComplianceChecker.run_checker(filelist,['cf'],3,'normal',output_filename='temp_report.json',output_format='json_new')

with open('temp_report.json', 'r') as f:
    reportdata = json.loads(f.read())

report_files = [k for k in reportdata.keys()]
report_tests = [k for k in reportdata[report_files[0]].keys()]
report_levels = ['high', 'medium', 'low']
scores= defaultdict(dict)

for t in report_tests:
    scores = report_test(reportdata, t, report_files, report_levels, scores)
    #print(scores[report_files[0]])

scores[report_files[0]]

Results for cf checks
1 files were checked

High priority results

Medium priority results

1 files failed:
§2.5 Variables: For the variable sfcWind the missing_value must be equal to the _FillValue

1 files failed:
§2.6 Attributes: §2.6.2 global attribute title should exist and be a non-empty string

1 files failed:
§2.6 Attributes: §2.6.1 Conventions global attribute does not contain "CF-1.8"

1 files failed:
§7.3 Cell Methods: sfcWind:cell_methods contains an invalid method: interpolation

1 files failed:
§7.3 Cell Methods: §7.3.3 sfcWind:cell_methods contains an interval specification that does not parse: "1H". Should be in format "interval: <number> <units>"

1 files failed:
§7.3 Cell Methods: §7.3.3 Invalid cell_methods keyword "method:" for variable sfcWind. Must be one of [interval, comment]

1 files failed:
§7.3 Cell Methods: §7.3.3 sfcWind:cell_methods contains an interval specification that does not parse: "1D". Should be in format "interval: <number> <units>"

Low priority 

{'cf': {'3': 'Errors',
  '2': 'Warnings',
  '1': 'Info',
  'scored_points': 124,
  'possible_points': 132}}

In [41]:
check_suite = CheckSuite()
check_suite.load_all_available_checkers()
ComplianceChecker.run_checker(filelist,['acdd'],3,'normal',output_filename='temp_report.json',output_format='json_new')

with open('temp_report.json', 'r') as f:
    reportdata = json.loads(f.read())

report_files = [k for k in reportdata.keys()]
report_tests = [k for k in reportdata[report_files[0]].keys()]
report_levels = ['high', 'medium', 'low']
scores= defaultdict(dict)

for t in report_tests:
    scores = report_test(reportdata, t, report_files, report_levels, scores)
    #print(scores[report_files[0]])

scores[report_files[0]]

Results for acdd checks
1 files were checked

High priority results

1 files failed:
Global Attributes: title not present

1 files failed:
Global Attributes: keywords not present

1 files failed:
Global Attributes: summary not present

1 files failed:
Global Attributes: Conventions does not contain 'ACDD-1.3'

1 files failed:
variable "sfcWind" missing the following attributes:: coverage_content_type

1 files failed:
variable "time" missing the following attributes:: long_name

Medium priority results

1 files failed:
Global Attributes: acknowledgment/acknowledgement not present

1 files failed:
Global Attributes: id not present

1 files failed:
Global Attributes: naming_authority not present

1 files failed:
Global Attributes: comment not present

1 files failed:
Global Attributes: creator_name not present

1 files failed:
Global Attributes: creator_url not present

1 files failed:
Global Attributes: creator_email not present

1 files failed:
Global Attributes: institution not present

{'acdd': {'3': 'Highly Recommended',
  '2': 'Recommended',
  '1': 'Suggested',
  'scored_points': 13,
  'possible_points': 53}}

In [42]:
qdcoutput=xr.open_dataset(filelist[0])
qdcoutput

<xarray.Dataset>
Dimensions:  (time: 365, lon: 1082, lat: 646)
Coordinates:
  * time     (time) datetime64[ns] 2035-01-01 2035-01-02 ... 2035-12-31
  * lon      (lon) float64 88.48 88.59 88.7 88.81 ... 207.1 207.2 207.3 207.4
  * lat      (lat) float64 -57.97 -57.86 -57.75 -57.64 ... 12.76 12.87 12.98
Data variables:
    sfcWind  (time, lat, lon) float64 ...
Attributes:
    CDI:          Climate Data Interface version 2.0.5 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    xclim:        [2024-02-19 16:43:17] : Bias-adjusted with QuantileDeltaMap...
    history:      Tue Feb 20 10:41:10 2024: cdo -z zip_5 -seldate,2035-01-01,...
    CDO:          Climate Data Operators version 2.0.5 (https://mpimet.mpg.de...

In [43]:
qdcoutput['sfcWind']

<xarray.DataArray 'sfcWind' (time: 365, lat: 646, lon: 1082)>
[255124780 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 2035-01-01 2035-01-02 ... 2035-12-31
  * lon      (lon) float64 88.48 88.59 88.7 88.81 ... 207.1 207.2 207.3 207.4
  * lat      (lat) float64 -57.97 -57.86 -57.75 -57.64 ... 12.76 12.87 12.98
Attributes:
    standard_name:  wind_speed
    long_name:      Near-Surface Wind Speed
    units:          m s-1
    cell_methods:   time: point (interval: 1H) area: interpolation (method: b...

In [14]:
qdcoutput['lon']

<xarray.DataArray 'lon' (lon: 1082)>
array([ 88.48,  88.59,  88.7 , ..., 207.17, 207.28, 207.39])
Coordinates:
  * lon      (lon) float64 88.48 88.59 88.7 88.81 ... 207.1 207.2 207.3 207.4
Attributes:
    standard_name:  longitude
    long_name:      longitude
    units:          degrees_east
    axis:           X

In [9]:
qdcoutput['time']

<xarray.DataArray 'time' (time: 365)>
array(['2035-01-01T00:00:00.000000000', '2035-01-02T00:00:00.000000000',
       '2035-01-03T00:00:00.000000000', ..., '2035-12-29T00:00:00.000000000',
       '2035-12-30T00:00:00.000000000', '2035-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2035-01-01 2035-01-02 ... 2035-12-31
Attributes:
    standard_name:  time
    axis:           T

In [10]:
barra_input=xr.open_dataset('/g/data/ob53/BARRA2/output/reanalysis/AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1/day/')

In [11]:
barra_input

<xarray.Dataset>
Dimensions:    (time: 31, lat: 646, lon: 1082, bnds: 2)
Coordinates:
  * time       (time) datetime64[ns] 1985-01-01T12:00:00 ... 1985-01-31T12:00:00
  * lat        (lat) float64 -57.97 -57.86 -57.75 -57.64 ... 12.76 12.87 12.98
  * lon        (lon) float64 88.48 88.59 88.7 88.81 ... 207.1 207.2 207.3 207.4
  * bnds       (bnds) float64 0.0 1.0
Data variables:
    pr         (time, lat, lon) float64 ...
    time_bnds  (time, bnds) datetime64[ns] ...
Attributes: (12/56)
    axiom_version:             0.1.0
    axiom_schemas_version:     0.1.0
    axiom_schema:              cordex-1D.json
    productive_version:        61832e0
    Conventions:               CF-1.10, ACDD-1.3
    activity_id:               reanalysis
    ...                        ...
    date_modified:             2023-11-17T03:05:59Z
    date_metadata_modified:    2023-11-17T03:05:59Z
    history:                   Thu Sep 28 13:22:08 2023: /g/data/access/ngm/m...
    references:                https://doi.org/10.25914/1x6g-2v48
    license:                   https://doi.org/10.25914/1x6g-2v48
    acknowledgement:           The production of BARRA2 was supported with fu...

In [12]:
barra_input['pr']

<xarray.DataArray 'pr' (time: 31, lat: 646, lon: 1082)>
[21668132 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 1985-01-01T12:00:00 ... 1985-01-31T12:00:00
  * lat      (lat) float64 -57.97 -57.86 -57.75 -57.64 ... 12.76 12.87 12.98
  * lon      (lon) float64 88.48 88.59 88.7 88.81 ... 207.1 207.2 207.3 207.4
Attributes:
    long_name:      Precipitation
    standard_name:  precipitation_flux
    units:          kg m-2 s-1
    cell_methods:   time: mean (interval: 1 hour) time: mean (interval: 1D)